In [ ]:
from blazingsql import BlazingContext
import pandas as pd

bc = BlazingContext()

In [ ]:
bc.s3('milestone',
     bucket_name='datalake',
     access_key_id='epapenhausen',
     secret_key='5d96af203bf43b74a93765dc',
     endpoint_override="http://130.245.177.209:9000")

#### Add U-Tables

In [ ]:
bc.create_table('u100', 's3://milestone/epapenhausen/Milestone/pdavies/u100.csv')
bc.create_table('u101', 's3://milestone/epapenhausen/Milestone/pdavies/u101.csv')
bc.create_table('u104', 's3://milestone/epapenhausen/Milestone/pdavies/u104.csv')
bc.create_table('u200', 's3://milestone/epapenhausen/Milestone/pdavies/u200.csv')
bc.create_table('u175', 's3://milestone/epapenhausen/Milestone/pdavies/u175.csv')
bc.create_table('u141', 's3://milestone/epapenhausen/Milestone/pdavies/u141.csv')
bc.create_table('u108', 's3://milestone/epapenhausen/Milestone/pdavies/u108.csv')
bc.create_table('u178', 's3://milestone/epapenhausen/Milestone/pdavies/u178.csv')
bc.create_table('u280', 's3://milestone/epapenhausen/Milestone/pdavies/u280.csv')
bc.create_table('u210', 's3://milestone/epapenhausen/Milestone/pdavies/u210.csv')
bc.create_table('u780', 's3://milestone/epapenhausen/Milestone/pdavies/u780_NAV-Movement.csv')

#### Add L-Tables

In [ ]:
bc.create_table('l220', 's3://milestone/epapenhausen/Milestone/pdavies/l220.csv')
bc.create_table('l222', 's3://milestone/epapenhausen/Milestone/pdavies/l222.csv')
bc.create_table('l225', 's3://milestone/epapenhausen/Milestone/pdavies/l225.csv')
bc.create_table('l235', 's3://milestone/epapenhausen/Milestone/pdavies/l235.csv')
bc.create_table('l295', 's3://milestone/epapenhausen/Milestone/pdavies/l295.csv')
bc.create_table('l315', 's3://milestone/epapenhausen/Milestone/pdavies/l315.csv')

#### Add P-Tables

In [ ]:
bc.create_table('p200', 's3://milestone/epapenhausen/Milestone/pdavies/p200.csv')
bc.create_table('p201', 's3://milestone/epapenhausen/Milestone/pdavies/p201.csv')
bc.create_table('p202', 's3://milestone/epapenhausen/Milestone/pdavies/p202.csv')
bc.create_table('p203', 's3://milestone/epapenhausen/Milestone/pdavies/p203.txt')
bc.create_table('p204', 's3://milestone/epapenhausen/Milestone/pdavies/p204.csv')
bc.create_table('p205', 's3://milestone/epapenhausen/Milestone/pdavies/p205.csv')
bc.create_table('p206', 's3://milestone/epapenhausen/Milestone/pdavies/p206.csv')
bc.create_table('p207', 's3://milestone/epapenhausen/Milestone/pdavies/p207.csv')
bc.create_table('p208', 's3://milestone/epapenhausen/Milestone/pdavies/p208.csv')
bc.create_table('p209', 's3://milestone/epapenhausen/Milestone/pdavies/p209.csv')
bc.create_table('p210', 's3://milestone/epapenhausen/Milestone/pdavies/p210.csv')

In [ ]:
u100 = bc.sql("select * from u100")
u101 = bc.sql("select * from u101")
u104 = bc.sql("select * from u104")
u200 = bc.sql("select * from u200")
u175 = bc.sql("select * from u175")
u141 = bc.sql("select * from u141")
u108 = bc.sql("select * from u108")
u178 = bc.sql("select * from u178")
u280 = bc.sql("select * from u280")
u210 = bc.sql("select * from u210")
u780 = bc.sql("select * from u780")

In [ ]:
p200 = bc.sql("select * from p200")
p201 = bc.sql("select * from p201")
p202 = bc.sql("select * from p202")
p203 = bc.sql("select * from p203")
p204 = bc.sql("select * from p204")
p205 = bc.sql("select * from p205")
p206 = bc.sql("select * from p206")
p207 = bc.sql("select * from p207")
p208 = bc.sql("select * from p208")
p209 = bc.sql("select * from p209")
p210 = bc.sql("select * from p210")

In [ ]:
l220 = bc.sql("select * from l220")
l222 = bc.sql("select * from l222")
l225 = bc.sql("select * from l225")
l235 = bc.sql("select * from l235")
l295 = bc.sql("select * from l295")
l315 = bc.sql("select * from l315")

In [ ]:
# Compound unique keys: (Entity Code, Effective Date, Run_ID, Price Run)
def sanitize(table, run_type='Oversight', status=None):
    filt = (table['price_run_type']==run_type)
    if status is not None:
        filt = filt&(table['p_status']==status)
    
    table_san = table[filt]
    drop = [c for c in table_san.columns if table_san[c].nunique() <= 1]+['pcontrol_id']
    return table_san.drop(drop, axis=1)  

In [ ]:
u101_san = sanitize(u101, status='V')
u100_san = sanitize(u100, status='V')

a01 = sanitize(p200)
a02 = sanitize(p201)
a03 = sanitize(p202)
a04 = sanitize(p203)

In [ ]:
def nested_summary(raw, keys, data_name):
    ''' Computes summary for categorical attributes using "keys" as a primary key. '''
    categories = [c for c in raw.columns[raw.dtypes==object] if c not in keys]
    cat_data = {str(cat)+'_'+str(level): [] for cat in categories for level in raw[cat].unique()}

    data = {**{k: [] for k in keys}, 'num_'+data_name: [], **cat_data}    
    for k, g in raw.groupby(keys): #'pcontrol_code'):
        n = g.shape[0]
        base_value = g.base_mv_close.sum()
        
        for cd in cat_data:
            data[cd].append(0)
                    
        for cat in categories:        
            #cat_vc = g[cat].value_counts()
            cat_vc = g.groupby(cat)['base_mv_close'].sum()
            for level, value in cat_vc.to_pandas().iteritems():
                index = str(cat)+'_'+str(level)                     
                data[index][-1] = value / base_value
                                
        for i, ky in enumerate(keys):
            data[ky].append(k[i])
        
        data['num_'+data_name].append(n)          
    
    return pd.DataFrame(data)

In [ ]:
keys = ['pcontrol_code', 'effective_date']
u100_data = nested_summary(u100_san, keys, 'assets')

In [ ]:
import pandas as pd
primary_key = ['pcontrol_code', 'effective_date']
product_data = pd.merge(a01.to_pandas(), u101_san.to_pandas(), how='left', left_on=primary_key, right_on=primary_key)
product_data = pd.merge(product_data, u100_data, how='left', left_on=primary_key, right_on=primary_key)

Specify columns to be dropped as they are not relevant to the analysis

In [ ]:
# drop date columns not relevant to the analysis
datedrop = ['last_updated_date_x', 'last_updated_date_y', 'run_date_and_time', 'value_date', 'effective_date']

# since we will be using the max_tol_lvl to construct the target variable, drop variables that could cause leakage
leakdrop = ['max_tol_lvl', 'holding_max_tol_lvl']

# drop columns indicating the data source
sourcedrop = [c for c in product_data.columns if '_source_' in c]

# drop columns relating to internal pcontrol flags
pcontroldrop = ['integrity_value', 'pcontrol_code'] + [c for c in product_data.columns if 'batch' in c] + [c for c in product_data.columns if c[:2] == 'c_'] + [c for c in product_data.columns if 'updated_by' in c]

drop = datedrop + leakdrop + sourcedrop + pcontroldrop

Extract the Year, Month, and Day from the date columns

In [ ]:
import numpy as np
value_year = product_data['value_date'].apply(lambda x: float(str(x)[:4]) if not np.isnan(x) else x)
value_month = product_data['value_date'].apply(lambda x: float(str(x)[4:6]) if not np.isnan(x) else x)
value_day = product_data['value_date'].apply(lambda x: float(str(x)[6:]) if not np.isnan(x) else x)

eff_year = product_data['effective_date'].apply(lambda x: float(str(x)[:4]) if not np.isnan(x) else x)
eff_month = product_data['effective_date'].apply(lambda x: float(str(x)[4:6]) if not np.isnan(x) else x)
eff_day = product_data['effective_date'].apply(lambda x: float(str(x)[6:]) if not np.isnan(x) else x)

In [ ]:
product_data['value_date_year'] = value_year
product_data['value_date_month'] = value_month
product_data['value_date_day'] = value_day

product_data['effective_date_year'] = eff_year
product_data['effective_date_month'] = eff_month
product_data['effective_date_day'] = eff_day

In [ ]:
product_data['pcontrol_code'].unique()

fund_names = u100_san['pcontrol_code'].unique()
product_data['fund'] = product_data['pcontrol_code'].apply(lambda x: next((f for f in fund_names if f in x), x))

#### A01 Product Validation

In [ ]:
import data_context_map.pattern_miner as pm

# A tolerance breach has occured if max_tol_lvl > 0
data = product_data.copy()
data['breached'] = (data['max_tol_lvl']>0).astype(int)

# drop the occurrences where the previous nav amount is 0
data = data[data['net_nav_amount_prev']!=0]
data.drop(drop, axis=1, inplace=True)

target = 'breached'
out = pm.DataContextMap(data, target, max_pattern=1000, max_depth=3, mine_type='binary', holdout=0)
out.render()